In [201]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn import svm
import nltk
import emoji
import re

In [171]:
data = pd.read_csv("Danish/Danish/offenseval-da-training-v1.tsv", sep='\t')[:-1]

texts = data["tweet"].apply(str)
labels = data["subtask_a"].apply(str)
print(data.loc[2883])
#for el in data["id"]:
 #   print(type(el))
texts

id                                                         264
tweet        @USER næste gang pastaen er brændt på ! 😂 det ...
subtask_a                                                  NOT
Name: 2883, dtype: object


0       Jeg tror det vil være dejlig køligt, men jeg v...
1       Så kommer de nok til at investere i en ny cyke...
2       Nu er det jo også de Ikea-aber der har lavet s...
3       128 Varme emails, er vi enige om at det er sex...
4       Desværre tyder det på, at amerikanerne er helt...
                              ...                        
2955    Har sgu lidt en anelse om... det her kunne mås...
2956    Ind og ruske tremmer med hende,Den syge kælling!!
2957                                             fedtmule
2958                                    ##HAR I HØRT DET?
2959    Kommer det bag på nogen? Det er jo fucking var...
Name: tweet, Length: 2960, dtype: object

In [173]:
#tokenization
texts_tokenized = []
for el in texts:
    texts_tokenized.append([emoji.demojize(el) for el in el.split()]) #nltk.word_tokenize(el)

    
texts_tokenized[2883]

['@USER',
 'næste',
 'gang',
 'pastaen',
 'er',
 'brændt',
 'på',
 '!',
 ':face_with_tears_of_joy:',
 'det',
 'jo',
 'sygt...']

In [186]:
#normalization
texts_normalized = []
for sent in texts_tokenized:
    # remove emojis
    sent = [el.lower() for el in sent if not bool(re.search(r":.*:", el))]
    # remove repeating characters
    sent = [re.sub(r"(.)\1{2,}", r"\1", el) for el in sent]
    texts_normalized.append(sent)
        
print(texts_normalized[2883])
#string = "heeeeello"
#s = re.sub(r"(.)\1{2,}", r"\1", string)
#print(texts_tokenized[2883])
#print([re.sub(r"(.)\1{2,}", r"\1", el) for el in texts_tokenized[2883]])
# print([el for el in texts_tokenized[2883] if not bool(re.search(r":.*:", el))])

['@user', 'næste', 'gang', 'pastaen', 'er', 'brændt', 'på', '!', 'det', 'jo', 'sygt.']


In [98]:
data

,id,tweet,subtask_a
0,3131,"Jeg tror det vil være dejlig køligt, men jeg v...",NOT
1,711,Så kommer de nok til at investere i en ny cyke...,NOT
2,2500,Nu er det jo også de Ikea-aber der har lavet s...,OFF
3,2678,"128 Varme emails, er vi enige om at det er sex...",NOT
4,784,"Desværre tyder det på, at amerikanerne er helt...",NOT
...,...,...,...
2955,170,Har sgu lidt en anelse om... det her kunne mås...,NOT
2956,1226,"Ind og ruske tremmer med hende,Den syge kælling!!",OFF
2957,2596,fedtmule,NOT
2958,1802,##HAR I HØRT DET?,NOT


In [190]:
texts_normalized2 = [" ".join(el) for el in texts_normalized]
texts_normalized3 = pd.DataFrame(texts_normalized2, columns=["tweet"])
texts_normalized3
data["tweet"] = texts_normalized2
data

,id,tweet,subtask_a
0,3131,"jeg tror det vil være dejlig køligt, men jeg v...",NOT
1,711,så kommer de nok til at investere i en ny cyke...,NOT
2,2500,nu er det jo også de ikea-aber der har lavet s...,OFF
3,2678,"128 varme emails, er vi enige om at det er sex...",NOT
4,784,"desværre tyder det på, at amerikanerne er helt...",NOT
...,...,...,...
2955,170,har sgu lidt en anelse om. det her kunne måske...,NOT
2956,1226,"ind og ruske tremmer med hende,den syge kælling!!",OFF
2957,2596,fedtmule,NOT
2958,1802,##har i hørt det?,NOT


In [191]:
texts = data["tweet"].apply(str)
texts, labels

(0       jeg tror det vil være dejlig køligt, men jeg v...
 1       så kommer de nok til at investere i en ny cyke...
 2       nu er det jo også de ikea-aber der har lavet s...
 3       128 varme emails, er vi enige om at det er sex...
 4       desværre tyder det på, at amerikanerne er helt...
                               ...                        
 2955    har sgu lidt en anelse om. det her kunne måske...
 2956    ind og ruske tremmer med hende,den syge kælling!!
 2957                                             fedtmule
 2958                                    ##har i hørt det?
 2959    kommer det bag på nogen? det er jo fucking var...
 Name: tweet, Length: 2960, dtype: object,
 0       NOT
 1       NOT
 2       OFF
 3       NOT
 4       NOT
        ... 
 2955    NOT
 2956    OFF
 2957    NOT
 2958    NOT
 2959    OFF
 Name: subtask_a, Length: 2960, dtype: object)

In [192]:
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.25, random_state=123)

In [193]:
texts_train, texts_test, labels_train, labels_test

(2734    jeg vil foreslå to andre billeder af københavn...
 2886    [og der bliver stadigt flere!](url## danmark l...
 2738                                                  lol
 2937    ah, og vi har allerede fået bygget en mur mod ...
 2671    vi må da sige, han går da all in når man skal ...
                               ...                        
 1147    det er jo ikke sjovt! nogen ting spøger man ba...
 2154                                  det har du os rat i
 1766                          grønt er altså for bornholm
 1122    de skal bare kvæles langsomt så de ligger stil...
 1346    syntes man begynder at høre mere om partering ...
 Name: tweet, Length: 2220, dtype: object,
 418     ikke noget andre kvinder ikke har men ser bedr...
 1224    foretrækker dog isterningebakken, den kan brug...
 1617    troede at vores flag altid brandte?.. det gør ...
 1745                                                mere?
 2139                      også kendt, som opium med brus.
             

In [195]:
vectorizer = TfidfVectorizer()
vectorizer.fit(texts)
print(vectorizer.idf_)

[7.60698797 8.30013515 7.60698797 ... 7.89467004 6.50837568 8.30013515]


In [196]:
vectors = vectorizer.transform(texts)
for el in vectors.toarray():
    print(el)

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0.

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0.

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0.

In [197]:
le = preprocessing.LabelEncoder()
le.fit(["NOT", "OFF"])
classes = le.transform(labels)
classes

array([0, 0, 1, ..., 0, 0, 1])

In [198]:
texts_train, texts_test, labels_train, labels_test = train_test_split(vectors.toarray(), classes, test_size=0.25, random_state=123)
texts_train, texts_test, labels_train, labels_test

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0,

In [199]:
clf = svm.SVC()
clf.fit(texts_train, labels_train)

SVC()

In [200]:
pred = clf.predict(texts_test)
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,